compared to the last version, this version added the imaging quality precheck of each trial, and active neuron selection no longer only select the sensory related neurons but also motor related neurons.

The threshold is:

* if the neuron is either a sensory related neuron or a motor related neuron, it would be selected as active neuron
    * sensory related neuron: the neuron is responsive to at least one kind of stimulus out of 3 stimulus(30% of trials coeff>0.35 and R2>0.36)
    * motor related neuron: the neuron is related to at least one kind of behavior(PC or freezing), if one kind of behavior have less than 4 trials, then if the neuron is a active neuron is it is responsive in at least one trial, or it need to be responsvie in more than 30% of trials.

In [ ]:
from attention_project_utils import * 
import numpy as np
import pandas as pd
import os
import tifffile
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
import matplotlib.lines as mlines
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform

import seaborn as sns


import sklearn
from sklearn import linear_model
from sklearn.metrics import r2_score 
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import silhouette_score

In [ ]:
def gen_regressor(x, tau, fps=3, record_trail=1):
    duration = int(length/(fps*record_trail)) *  (record_trail)
    print(length)
    t = np.arange(0, duration, duration/len(x))
    print(t.shape)
    if tau == 0:
        return x
    zeros = np.zeros_like(t)
    print(zeros.shape)
    padded = np.concatenate([zeros,x,zeros])
#     plt.plot(padded)
#     plt.show()
    kernel = np.concatenate([zeros, np.exp(-t / tau )])
    print( np.exp(-t / tau ))
#     plt.plot(kernel)
#     plt.show()
    convolved = np.convolve(padded, kernel, mode='same')[len(t):-len(t)]    
    if convolved.mean() == 0:
        return convolved
    return convolved / convolved.mean()

def get_regression(df0f):
    listregintercept=[]
    listregcoeff=[]
    listr2=[] 
    
#     regressors = {'Dot': a, 'OKR': b, 'Loom': c}
    regressors = {'Dot': a}
#     print(regressors)
    X = pd.DataFrame(regressors,columns=['Dot']) 
#     print(X)
   
    for i in range(0,len(df0f)):
        df = pd.DataFrame (df0f[i])     
        regr = linear_model.LinearRegression()           
        x = regr.fit(X,df)
        listregcoeff.append(regr.coef_[0][0])  
        listregintercept.append(x.intercept_[0])
        r2 = r2_score(df, x.predict(X), multioutput='raw_values')
        listr2.append(r2[0])  
    
    return listregintercept,listregcoeff,listr2

In [ ]:
list_calciumtraces = list(np.ones(39))
# length =len(list_calciumtraces[0])
length =len(list_calciumtraces)
# listprey,listokr,listloom = get_carbox(length)
listprey = np.concatenate([np.zeros(27),np.ones(3),np.zeros(9)])
# listokr = list(range(300,400))
# listloom = list(range(1700,1800))
print(listprey.shape)

a= gen_regressor(listprey,1.7)
a = (a-min(a))/(max(a)-min(a))
# a = np.concatenate([np.zeros(92),a,a,a,a,a,a])
# b= gen_regressor(listokr,2.6)
# b = (b-min(b))/(max(b)-min(b))

# c= gen_regressor(listloom,2.6)
# c = (c-min(c))/(max(c)-min(c))

# plt.axis('off')
plt.figure(dpi = 800)
plt.plot(a,color='navy',linewidth = 3)
plt.axis('off')
plt.show()

In [ ]:
plt.figure(dpi = 800)

plt.plot(a,color='navy',linewidth = 3)
plt.fill_between(np.arange(27,30),1,0,linewidth = 0,color = 'violet',alpha = 0.3)
plt.axis('off')
plt.show()

In [ ]:
#### main_dir_list = ["D:\\LCr_UV_HSA_Luminance_4\\20230102"]+["D:\\LCr_UV_HSA_Luminance_5\\20230109"]*2
# fish_index_list =['F2_16_00','F3_20_30','F4_22_30']

# main_dir_list = ["F:\\LCr_UV_HSA_Lum_Low_1\\20230312"]*5+["F:\\LCr_UV_HSA_Lum_Low_2\\20230409"]*3+["F:\\LCr_UV_HSA_Lum_Low_2\\20230410"]*4
# fish_index_list = ['F1_10_40','F2_12_20','F4_16_00','F5_18_00','F6_20_00','F2_13_00','F4_17_00','F5_19_00','F1_11_00','F2_13_00','F4_16_00','F6_19_30']

main_dir_list = ["F:\\LCr_UV_HSA_Lum_Low_1\\20230312"]*5+["F:\\LCr_UV_HSA_Lum_Low_2\\20230409"]*3+["F:\\LCr_UV_HSA_Lum_Low_2\\20230410"]*2
fish_index_list = ['F1_10_40','F2_12_20','F4_16_00','F5_18_00','F6_20_00','F2_13_00','F4_17_00','F5_19_00','F2_13_00','F4_16_00']

visuomotor_list = ['ns','Prey','Freezing','Luminance','motor_Freezing','motor_PC']

time_len = 39
plane_no = 9

output_dir = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_low_elevation'
create_directory(output_dir)

df_reg_clusters = pd.read_csv(output_dir+"\\low_elev_registered_10fish_clustering_withfish_withROIindex.csv")
df_VMV_all = pd.read_csv(output_dir+'\\df_VMV_10fish_v5.csv')



In [ ]:
# filter out the cell bodies who are in PVN
PVN_mask = tifffile.imread('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\mapzebrain_brain_regions\\periventricular_layer.tif')
mask_xy_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xy.tif'
mask_xy = tifffile.imread(mask_xy_path)
mask_xz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xz.tif'
mask_xz = tifffile.imread(mask_xz_path)
mask_yz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_yz.tif'
mask_yz = tifffile.imread(mask_yz_path)

X = df_reg_clusters.x/0.994
Y = df_reg_clusters.y/0.994
Z = df_reg_clusters.z

filtered_idx = []
for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
    if PVN_mask[int(z),int(y),int(x)] == 255:
        filtered_idx.append(i)
        

df_VMV_PVN  = df_VMV_all.loc[filtered_idx,:]

npy_VMV_PVN = df_VMV_PVN.iloc[:,:6].to_numpy(dtype = 'float64')

corr = np.corrcoef(npy_VMV_PVN)
plt.imshow(corr)
plt.show()

In [ ]:
d = sch.distance.pdist(corr)
L = sch.linkage(d, method='complete')

In [ ]:
d = sch.distance.pdist(corr)
L = sch.linkage(d, method='average')

In [ ]:
test = sch.linkage(corr, method='complete')

In [ ]:
def sil_scores(X, Z, ts):
    '''
    Creates a list of silhouette scores for a predetermined range of clusters.
    '''
    scores = []
    
    for num_clust in ts:
        print(num_clust)
        scores.append(silhouette_score(X, fcluster(Z, t=num_clust, criterion='maxclust')))
    
    return scores

scores = sil_scores(corr, L, range(2,70))


# for t, s in zip(range(2,70), scores):
#     print(f"With {t} clusters, the silhouette score is {s}")

In [ ]:
scores

In [ ]:
plt.plot(range(2,70),scores)
plt.xlabel('# of cluster',fontsize = 15)
plt.ylabel('Silhouette Coefficient',fontsize = 15)
plt.axvline(x= 8,color = 'grey')
plt.annotate('cluster No. = 8',(8.8,0.3),fontsize = 15)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.title('PVN clustering silhouette analysis',fontsize  =18)
plt.show()

In [ ]:
# 48 clusters
threshold_ratio = 0.4
clusters = sch.fcluster(L, threshold_ratio * d.max(), 'distance')
argsort = np.argsort(clusters)[::-1]
npy_VMV_PVN_cluster = np.c_[npy_VMV_PVN, clusters]

In [ ]:
# 8 clusters
clusters = fcluster(L, 8, 'maxclust')
argsort = np.argsort(clusters)[::-1]
npy_VMV_PVN_cluster = np.c_[npy_VMV_PVN, clusters]

In [ ]:
# 3 clusters
clusters = fcluster(L, 2, 'maxclust')
argsort = np.argsort(clusters)[::-1]
npy_VMV_PVN_cluster = np.c_[npy_VMV_PVN, clusters]

In [ ]:
# 10 clusters
clusters = fcluster(L, 6, 'maxclust')
argsort = np.argsort(clusters)[::-1]
npy_VMV_PVN_cluster = np.c_[npy_VMV_PVN, clusters]

In [ ]:
np.unique(clusters)

In [ ]:
# vmv of different clusters

fig,[ax2,ax1,ax3]= plt.subplots(1,3,figsize = (18,14), gridspec_kw=dict(width_ratios=[.07, .85, .08]),dpi = 900)

# fig = plt.figure(figsize = (12,14))
# ax2 = plt.subplot(131, gridspec_kw=dict(width_ratios=[.07, .85, .08]))
# ax1 = plt.subplot(132)
# ax3 = plt.subplot(235)
# d= sch.dendrogram(L, orientation='left', ax=ax4, color_threshold=threshold_ratio*d.max())

argsort = np.argsort(clusters)[::-1]
sns.heatmap(npy_VMV_PVN_cluster[argsort,:6],cmap = 'plasma',ax = ax1,vmin = -0.3,vmax = 0.7,cbar_ax = ax3)
sns.heatmap(npy_VMV_PVN_cluster[argsort,-1:],cmap = 'gist_ncar',cbar = False,ax = ax2)

ax1.set_yticks([]) 
ax1.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 19)
for i in range(0,4):
    ax1.get_xticklabels()[i].set_color("maroon")
for i in range(4,6):
    ax1.get_xticklabels()[i].set_color("navy")
    
# ax2.set_ylabel('Neurons',fontsize = 18) 
ax2.set_yticks([]) 
ax2.set_xticklabels(['Cluster ID'],fontsize = 20)


line_x = np.where(clusters[argsort][:-1]-clusters[argsort][1:] ==1)
ax1.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)
ax2.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)


fig.subplots_adjust(wspace=0.02)

ax1.set_xlabel('Visuomotor Vectors',fontsize = 22)

ax3.set_ylabel('Averaged Visomotor Vector Score',fontsize = 24)
ax3.tick_params(labelsize=22)



# ax4.axis('off')

plt.show()

In [ ]:
# 4deg and 15deg sensory make a normalization
npy_VMV_PVN_cluster_normed = npy_VMV_PVN_cluster.copy()
# VMV_max = np.max(npy_VMV_PVN_cluster_normed[:,:6])
# VMV_min = np.min(npy_VMV_PVN_cluster_normed[:,:6])
# npy_VMV_PVN_cluster_normed[:,1] = (npy_VMV_PVN_cluster_normed[:,1]-VMV_min)/(VMV_max-VMV_min)
# npy_VMV_PVN_cluster_normed[:,2] = (npy_VMV_PVN_cluster_normed[:,2]-VMV_min)/(VMV_max-VMV_min)

In [ ]:
# 2d scatter plot of distribution of each cluster
fig,ax1 = plt.subplots(1,1,figsize = (4,3),dpi=800)
cluster_idx=np.unique(clusters)
startcolor = 'crimson'
midcolor = '#FDFD96'
endcolor = 'dodgerblue'
cm = LinearSegmentedColormap.from_list( 'own2', [startcolor, midcolor, endcolor] )
scatter_info = np.zeros((len(cluster_idx),4))

color_cluster = np.zeros(npy_VMV_PVN_cluster.shape[0])

for cluster_i in range(0,len(cluster_idx)):
    i = int(cluster_idx[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
    average_temp_cluster = np.average(npy_VMV_PVN_cluster_normed[locals()['cluster_'+str(i)+'_roi_idx'],:6],axis = 0)
    scatter_info[cluster_i,0] = average_temp_cluster[4]
    scatter_info[cluster_i,1] = average_temp_cluster[5]
    scatter_info[cluster_i,2] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    color_cluster[locals()['cluster_'+str(i)+'_roi_idx']] = scatter_info[cluster_i,2]
#     scatter_info[cluster_i,2] = (average_temp_cluster[2]-average_temp_cluster[1])
    scatter_info[cluster_i,3] = len(locals()['cluster_'+str(i)+'_roi_idx'])
scatter_info[:,3] = scatter_info[:,3]/np.sum(scatter_info[:,3])
for cluster_i in range(0,len(cluster_idx)):
    i = int(cluster_idx[cluster_i])
    average_temp_cluster = np.average(npy_VMV_PVN_cluster_normed[locals()['cluster_'+str(i)+'_roi_idx'],:6],axis = 0)
    if scatter_info[cluster_i,3]>0.04:
        ax1.annotate(str(i),(average_temp_cluster[4]-0.011,average_temp_cluster[5]-0.007),fontsize = 8)
        
#     if scatter_info[cluster_i,0]>0.08 and scatter_info[cluster_i,1]>0.08:
#         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
#         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'both'
#     elif scatter_info[cluster_i,0]<scatter_info[cluster_i,1]:
#         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
#         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'PC'
#     else:
#         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
#         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'Freezing'

sc = plt.scatter(scatter_info[:,0],scatter_info[:,1],c = scatter_info[:,2],s = 1200*scatter_info[:,3],cmap = cm,vmin = -1,vmax = 1,alpha = 0.8)
sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both')
sc_cbar.set_label(label='Size Preference',size=12,labelpad = -6)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
# sc_cbar,'Prefer 4°,'Prefer 15°'
plt.xlabel('Freezing (motor) score',fontsize = 12)
plt.ylabel('Prey (motor) score',fontsize = 12)
plt.xlim(-0.06,0.36)
plt.ylim(-0.06,0.36)
plt.xticks([0,0.1,0.2,0.3])
plt.yticks([0,0.1,0.2,0.3])
ax1.axhline(y = 0.08,xmin = 0.48,xmax = 0.8,color = 'dodgerblue',linewidth =2)
ax1.axhline(y = -0.05,xmin = 0.48,xmax = 0.8,color ='dodgerblue',linewidth =2)
ax1.axvline(x = 0.14,ymin = 0.1/4.2,ymax = 1.4/4.2,color = 'dodgerblue',linewidth =2)
ax1.axvline(x = 0.36-0.2*0.42,ymin = 0.1/4.2,ymax = 1.4/4.2,color = 'dodgerblue',linewidth =2)

ax1.axvline(x = 0.08,ymin = 0.48,ymax = 0.8,color = 'crimson',linewidth =2)
ax1.axvline(x = -0.05,ymin = 0.48,ymax = 0.8,color = 'crimson',linewidth =2)
ax1.axhline(y = 0.14,xmin = 0.1/4.2,xmax = 1.4/4.2,color = 'crimson',linewidth =2)
ax1.axhline(y = 0.36-0.2*0.42,xmin = 0.1/4.2,xmax = 1.4/4.2,color = 'crimson',linewidth =2)

# plt.plot([0.08,0.08],[0.08,1],linestyle='dashed',color = 'k')
# plt.plot([0.08,1],[0.08,0.08],linestyle='dashed',color = 'k')
# plt.title('Cluster Distribution(HSA neurons)')
ax1.tick_params(direction='out', length=4, width=1)
plt.show()

In [ ]:
# vmv of different clusters

fig,[ax2,ax1,ax3]= plt.subplots(1,3,figsize = (18,14), gridspec_kw=dict(width_ratios=[.07, .85, .08]),dpi = 900)

# fig = plt.figure(figsize = (12,14))
# ax2 = plt.subplot(131, gridspec_kw=dict(width_ratios=[.07, .85, .08]))
# ax1 = plt.subplot(132)
# ax3 = plt.subplot(235)
# d= sch.dendrogram(L, orientation='left', ax=ax4, color_threshold=threshold_ratio*d.max())

argsort = np.argsort(color_cluster)[::-1]
reshape_color_cluster = color_cluster.reshape((color_cluster.shape[0],1))
sns.heatmap(npy_VMV_PVN_cluster[argsort,:6],cmap = 'plasma',ax = ax1,vmin = -0.3,vmax = 0.7,cbar_ax = ax3)
sns.heatmap(reshape_color_cluster[argsort,:],cmap = cm,vmin = -1,vmax = 1,cbar = False,ax = ax2)

ax1.set_yticks([]) 
ax1.set_xticklabels(['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)'],fontsize = 19)
for i in range(0,4):
    ax1.get_xticklabels()[i].set_color("maroon")
for i in range(4,6):
    ax1.get_xticklabels()[i].set_color("navy")
    
# ax2.set_ylabel('Neurons',fontsize = 18) 
ax2.set_yticks([]) 
ax2.set_xticklabels(['Cluster ID'],fontsize = 20)


line_x = np.where(reshape_color_cluster[argsort][:-1]-reshape_color_cluster[argsort][1:] !=0)
ax1.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)
ax2.hlines(line_x , *ax1.get_xlim(),color = 'white',linewidth = 1.5)


fig.subplots_adjust(wspace=0.02)

ax1.set_xlabel('Visuomotor Vectors',fontsize = 22)

ax3.set_ylabel('Averaged Visomotor Vector Score',fontsize = 24)
ax3.tick_params(labelsize=22)



# ax4.axis('off')

plt.show()

In [ ]:
# 2d scatter plot of distribution of each cluster
fig,ax1 = plt.subplots(1,1,figsize = (4,3),dpi=800)
cluster_idx=np.unique(clusters)
startcolor = 'crimson'
midcolor = '#FDFD96'
endcolor = 'dodgerblue'
cm = LinearSegmentedColormap.from_list( 'own2', [startcolor, midcolor, endcolor] )
# scatter_info = np.zeros((len(cluster_idx),4))
# for cluster_i in range(0,len(cluster_idx)):
#     i = int(cluster_idx[cluster_i])
#     locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
#     average_temp_cluster = np.average(npy_VMV_PVN_cluster_normed[locals()['cluster_'+str(i)+'_roi_idx'],:6],axis = 0)
#     scatter_info[cluster_i,0] = average_temp_cluster[4]
#     scatter_info[cluster_i,1] = average_temp_cluster[5]
#     scatter_info[cluster_i,2] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
# #     scatter_info[cluster_i,2] = (average_temp_cluster[2]-average_temp_cluster[1])
#     scatter_info[cluster_i,3] = len(locals()['cluster_'+str(i)+'_roi_idx'])
# scatter_info[:,3] = scatter_info[:,3]/np.sum(scatter_info[:,3])
# for cluster_i in range(0,len(cluster_idx)):
#     i = int(cluster_idx[cluster_i])
#     average_temp_cluster = np.average(npy_VMV_PVN_cluster_normed[locals()['cluster_'+str(i)+'_roi_idx'],:6],axis = 0)
#     if scatter_info[cluster_i,3]>0.04:
#         ax1.annotate(str(i),(average_temp_cluster[4]-0.0055,average_temp_cluster[5]-0.005),fontsize = 5)
        
# #     if scatter_info[cluster_i,0]>0.08 and scatter_info[cluster_i,1]>0.08:
# #         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
# #         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'both'
# #     elif scatter_info[cluster_i,0]<scatter_info[cluster_i,1]:
# #         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
# #         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'PC'
# #     else:
# #         index_temp = df_VMV_filtered.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:].index
# #         df_VMV_filtered.loc[index_temp,'cluster_type'] = 'Freezing'

sc = plt.scatter(npy_VMV_PVN_cluster_normed[:,4],npy_VMV_PVN_cluster_normed[:,5],c = (npy_VMV_PVN_cluster_normed[:,2]-npy_VMV_PVN_cluster_normed[:,1])/(npy_VMV_PVN_cluster_normed[:,2]+npy_VMV_PVN_cluster_normed[:,1]),s = 1,alpha = 0.7,cmap = cm,vmin = -0.5,vmax = 0.5)
sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both')
sc_cbar.set_label(label='Size Preference',size=12,labelpad = -6)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
# sc_cbar,'Prefer 4°,'Prefer 15°'
plt.xlabel('Freezing (motor) score',fontsize = 12)
plt.ylabel('Prey (motor) score',fontsize = 12)
plt.xlim(-0.06,0.36)
plt.ylim(-0.06,0.36)
plt.xticks([0,0.1,0.2,0.3])
plt.yticks([0,0.1,0.2,0.3])
# ax1.axhline(y = 0.1,xmin = 0.5,xmax = 0.8,color = 'dodgerblue',linewidth =2)
# ax1.axhline(y = -0.05,xmin = 0.5,xmax = 0.8,color ='dodgerblue',linewidth =2)
# ax1.axvline(x = 0.15,ymin = 0.1/4.2,ymax = 1.6/4.2,color = 'dodgerblue',linewidth =2)
# ax1.axvline(x = 0.36-0.2*0.42,ymin = 0.1/4.2,ymax = 1.6/4.2,color = 'dodgerblue',linewidth =2)

# ax1.axvline(x = 0.1,ymin = 0.5,ymax = 0.8,color = 'crimson',linewidth =2)
# ax1.axvline(x = -0.05,ymin = 0.5,ymax = 0.8,color = 'crimson',linewidth =2)
# ax1.axhline(y = 0.15,xmin = 0.1/4.2,xmax = 1.6/4.2,color = 'crimson',linewidth =2)
# ax1.axhline(y = 0.36-0.2*0.42,xmin = 0.1/4.2,xmax = 1.6/4.2,color = 'crimson',linewidth =2)

# plt.plot([0.08,0.08],[0.08,1],linestyle='dashed',color = 'k')
# plt.plot([0.08,1],[0.08,0.08],linestyle='dashed',color = 'k')
# plt.title('Cluster Distribution(HSA neurons)')
ax1.tick_params(direction='out', length=4, width=1)
plt.show()

In [ ]:
df_filtered_reg_clusters  = df_reg_clusters.loc[filtered_idx,:]
df_filtered_reg_clusters['cluster'] = clusters

In [ ]:
df_VMV_PVN['cluster'] = clusters
df_VMV_PVN.to_csv(output_dir+'\\df_VMV_PVN_48cluster_v6.csv',index=False)

In [ ]:
cluster_idx=np.unique(clusters)
for i in cluster_idx:
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])
    fig,ax1 = plt.subplots(1,1,figsize = (17,3))
    sns.heatmap(npy_VMV_PVN_cluster[locals()['cluster_'+str(i)+'_roi_idx'],:6],cmap = 'plasma',ax = ax1,vmin = -0.3,vmax = 0.7)
    ax1.set_xticklabels(visuomotor_list,fontsize = 14)
    for j in range(0,4):
        ax1.get_xticklabels()[j].set_color("maroon")
    for k in range(4,6):
        ax1.get_xticklabels()[k].set_color("navy")
    ax1.set_yticks([]) 
    plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 16)
    plt.show()
    
    

In [ ]:
atlas_path = 'C:\\Users\\yxton\\Documents\\Code\\registration_code_HSA\\T_AVG_H2BGCaMP.tif'
atlas = tifffile.imread(atlas_path)
mask_xy_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xy.tif'
mask_xy = tifffile.imread(mask_xy_path)
mask_xz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_xz.tif'
mask_xz = tifffile.imread(mask_xz_path)
mask_yz_path = 'C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files\\Mask_yz.tif'
mask_yz = tifffile.imread(mask_yz_path)

In [ ]:
df_filtered_reg_clusters  = df_reg_clusters.loc[filtered_idx,:]
df_filtered_reg_clusters['cluster'] = clusters

In [ ]:


xy_cropping = [200,559,0,597]
xz_cropping = [0,359,0,597]
yz_cropping = [0,359,0,597]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]

    
fig,axes = plt.subplots(1, 3,figsize = (9,3),subplot_kw=dict(box_aspect=3.59/5.97),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

size = 1

# colorcode = '#50d4f0'
# colorcode = 'lawngreen'
# colorcode = 'deepskyblue'
colorcode = 'dodgerblue'

# freezing_cluster_id_list = [6,8]
# # freezing_cluster_id_list = [2]
# for temp_cluster_id in freezing_cluster_id_list:
#     X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
#     Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
#     Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
#     filtered_idx = []
#     for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
#         if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
#             filtered_idx.append(i)

#     axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.6)
#     axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)
#     axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)

pc_cluster_id_list = [4,5]
# pc_cluster_id_list = [9]
# colorcode = '#fc5b44'
# colorcode = 'magenta'
colorcode = 'crimson'

for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.3)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.3)
    axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.3)


# 



axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=18, ha='center', va='baseline')

fig.tight_layout()
plt.show()


In [ ]:
# D/V A/V distribution of different clusters

freezing_cluster_id_list = [6,8]
# freezing_cluster_id_list = [2]
for temp_cluster_id in freezing_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
freezing_X = np.array(X)[filtered_idx]
freezing_Y = np.array(Y)[filtered_idx]
freezing_Z = 358-np.array(Z)[filtered_idx]

pc_cluster_id_list = [4,5]
for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
pc_X = np.array(X)[filtered_idx]
pc_Y = np.array(Y)[filtered_idx]
pc_Z = 358-np.array(Z)[filtered_idx]


sns.kdeplot(pc_Z,color = 'crimson')
sns.kdeplot(freezing_Z,color = 'dodgerblue')
plt.ylabel('KDE density',fontsize = 15)
plt.xlabel('Microns',fontsize = 15)
plt.yticks(fontsize = 15)
plt.xticks([0,50,100,150,200],['0(Dorsal)','50','100','150','200(Ventral)'],fontsize = 15)

plt.show()

sns.kdeplot(pc_Y*0.994,color = 'crimson')
sns.kdeplot(freezing_Y*0.994,color = 'dodgerblue')
plt.ylabel('KDE density',fontsize = 15)
plt.xlabel('Microns',fontsize = 15)
plt.yticks(fontsize = 15)
plt.xticks([200,300,400,500,600],['200(Anterior)','300','400','500','600(Posterior)'],fontsize = 15)

plt.show()

In [ ]:


xy_cropping = [200,559,0,597]
xz_cropping = [0,359,0,597]
yz_cropping = [0,359,0,597]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]

    
fig,axes = plt.subplots(1, 3,figsize = (9,3),subplot_kw=dict(box_aspect=3.59/5.97),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

size = 1

# colorcode = '#50d4f0'
# colorcode = 'lawngreen'
# colorcode = 'deepskyblue'
colorcode = 'dodgerblue'

freezing_cluster_id_list = [44,41]
for temp_cluster_id in freezing_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)

    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)

pc_cluster_id_list = [16]
# colorcode = '#fc5b44'
# colorcode = 'magenta'
colorcode = 'crimson'

for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.3)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.3)
    axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.3)


# 



axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=18, ha='center', va='baseline')

fig.tight_layout()
plt.show()


In [ ]:
# cropped version

xy_cropping = [200,559,60,517]
xz_cropping = [0,359,60,517]
yz_cropping = [0,359,120,577]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]

    
fig,axes = plt.subplots(1, 3,figsize = (8,3),subplot_kw=dict(box_aspect=3.59/4.57),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

size = 1

# colorcode = '#50d4f0'
# colorcode = 'lawngreen'
# colorcode = 'deepskyblue'
colorcode = 'dodgerblue'

freezing_cluster_id_list = [44,41]
for temp_cluster_id in freezing_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)

    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[2].scatter(np.array(Y)[filtered_idx]-yz_cropping[2],358-np.array(Z)[filtered_idx]-yz_cropping[0],color = colorcode,s = size,alpha = 0.6)

pc_cluster_id_list = [16]
# colorcode = '#fc5b44'
# colorcode = 'magenta'
colorcode = 'crimson'

for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.3)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.3)
    axes[2].scatter(np.array(Y)[filtered_idx]-yz_cropping[2],358-np.array(Z)[filtered_idx]-yz_cropping[0],color = colorcode,s = size,alpha = 0.3)


# 



axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=15, ha='center', va='baseline')
axes[1].axhline(y = 340,xmin = 0.8,xmax = 0.8+30.18/457,color = 'white',linewidth = 2)

fig.tight_layout()
plt.show()


In [ ]:
# D/V A/V distribution of different clusters

freezing_cluster_id_list = [44,41]

# freezing_cluster_id_list = [2]
for temp_cluster_id in freezing_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
freezing_X = np.array(X)[filtered_idx]
freezing_Y = np.array(Y)[filtered_idx]
freezing_Z = 358-np.array(Z)[filtered_idx]

pc_cluster_id_list = [16]
for temp_cluster_id in pc_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)
pc_X = np.array(X)[filtered_idx]
pc_Y = np.array(Y)[filtered_idx]
pc_Z = 358-np.array(Z)[filtered_idx]

fig,ax = plt.subplots(1,1,figsize = (5,2.4),dpi =800)
sns.kdeplot(pc_Z,color = 'crimson',linewidth = 3)
sns.kdeplot(freezing_Z,color = 'dodgerblue',linewidth = 3)
plt.ylabel('Density',fontsize = 12,labelpad = 0)
plt.xlabel('Microns',fontsize = 12)
plt.xticks([0,50,100,150,200],['0(D)','50','100','150','200(V)'],fontsize = 12)
plt.ylim(0,0.02)
plt.yticks([0,0.01,0.02],fontsize = 11)
plt.show()
fig,ax = plt.subplots(1,1,figsize = (5,2.4),dpi =800)
sns.kdeplot(pc_Y*0.994,color = 'crimson',linewidth = 3)
sns.kdeplot(freezing_Y*0.994,color = 'dodgerblue',linewidth = 3)
plt.ylabel('Density',fontsize = 12,labelpad = 0)
plt.xlabel('Microns',fontsize = 12)
plt.ylim(0,0.0115)
plt.yticks([0,0.005,0.01],fontsize = 11)
plt.xticks([200,300,400,500,600],['200(A)','300','400','500','600(P)'],fontsize = 12)

plt.show()

In [ ]:
# both cluster

xy_cropping = [200,559,0,597]
xz_cropping = [0,359,0,597]
yz_cropping = [0,359,0,597]

xy = np.mean(atlas,axis = 0)
yz = np.mean(atlas,axis = 2)
xz = np.mean(atlas,axis = 1)
rotated_yz = np.zeros((yz.shape[0],yz.shape[1]))
rotated_xz = np.zeros((xz.shape[0],xz.shape[1]))

for z in range(0,yz.shape[0]):
    rotated_yz[z,:] = yz[yz.shape[0]-1-z,:]
    rotated_xz[z,:] = xz[xz.shape[0]-1-z,:]

    
fig,axes = plt.subplots(1, 3,figsize = (9,3),subplot_kw=dict(box_aspect=3.59/5.97),dpi=800)

ax = sns.heatmap(xy[xy_cropping[0]:xy_cropping[1],xy_cropping[2]:xy_cropping[3]],ax = axes[0],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_xz[xz_cropping[0]:xz_cropping[1],xz_cropping[2]:xz_cropping[3]],ax = axes[1],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)
ax = sns.heatmap(rotated_yz[yz_cropping[0]:yz_cropping[1],yz_cropping[2]:yz_cropping[3]],ax = axes[2],cmap = 'gray',cbar = False,vmin = 0,vmax = 30)

size = 1

# colorcode = '#50d4f0'
# colorcode = 'lawngreen'
# colorcode = 'deepskyblue'
colorcode = '#FFCC00'

both_cluster_id_list = [22]
for temp_cluster_id in both_cluster_id_list:
    X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
    Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
    Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z
    filtered_idx = []
    for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
        if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
            filtered_idx.append(i)

    axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)
    axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.6)

# pc_cluster_id_list = [16]
# # colorcode = '#fc5b44'
# # colorcode = 'magenta'
# colorcode = 'crimson'

# for temp_cluster_id in pc_cluster_id_list:
#     X = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].x/0.994
#     Y = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].y/0.994
#     Z = df_filtered_reg_clusters[df_filtered_reg_clusters['cluster']==temp_cluster_id].z

#     filtered_idx = []
#     for x,y,z,i in zip(X,Y,Z,range(0,X.shape[0])):
#         if mask_xy[int(y),int(x)] == 255 and mask_xz[int(z),int(y)] == 255 and mask_yz[int(z),int(x)] == 255:
#             filtered_idx.append(i)
#     axes[0].scatter(np.array(X)[filtered_idx]-xy_cropping[2],np.array(Y)[filtered_idx]-xy_cropping[0],color = colorcode,s = size,alpha = 0.35)
#     axes[1].scatter(np.array(X)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.35)
#     axes[2].scatter(np.array(Y)[filtered_idx]-xz_cropping[2],358-np.array(Z)[filtered_idx]-xz_cropping[0],color = colorcode,s = size,alpha = 0.35)






axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')

cols = ['Horizontal','Coronal','Sagittal']
pad = 5 
for ax, col in zip(axes, cols):
    ax.annotate(col, xy=(0.5, 1.04), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size=18, ha='center', va='baseline')

fig.tight_layout()
plt.show()


In [ ]:
df_filtered_reg_clusters

In [ ]:
# normed score

df_active_PVN_normed_ratio_new = pd.DataFrame()
for main_dir,fish in zip(main_dir_list,fish_index_list):
    print(fish)
    date_no = main_dir.split('\\')[-1]
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish+'//vsinfo_'+date_no+'_'+fish+'.xlsx')
    if main_dir == "F:\\LCr_UV_HSA_Lum_Low_1\\20230312":
        bradyinfo_dir = glob.glob(main_dir+'\\bradyinfo\\Bradyinfo*'+fish+'.xlsx')[0]
    else:
        bradyinfo_dir = glob.glob(main_dir+'\\behavior\\'+fish+'\\Bradyinfo*'+fish+'.xlsx')[0]
 
    Brady_info = pd.read_excel(bradyinfo_dir,sheet_name = 'Bradyinfo')
    trial_no = vsinfo.shape[0]
    
    Index_nm_Freezing = []
    Index_nm_PC = []
    Index_nb_Freezing = []
    Index_nb_PC = []
    Index_Freezing = []
    Index_Prey = []
    
    for t in range(0,trial_no):
#         if Bad_Imaging_Quality_list.count(t) == 0:
        if vsinfo.Left_Stimulus_Type[t] != 'n' or vsinfo.Right_Stimulus_Type[t] != 'n':
            if Brady_info['Behavior_Classification'][t] == 'Freezing':
                Index_nm_Freezing.append(t)
            if Brady_info['Bradycardia_Classification'][t] == 0 and Brady_info['PC_trial'][t] == True:
                Index_nm_PC.append(t)
            if vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 20:
                    Index_Freezing.append(t)
                    if Brady_info['Behavior_Classification'][t] == 'Freezing':
                        Index_nb_Freezing.append(t)
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 4:
                    Index_Prey.append(t)
                    if Brady_info['Bradycardia_Classification'][t] == 0 and Brady_info['PC_trial'][t] == True:
                        Index_nb_PC.append(t)
    RSM_freezing = 1/(len(Index_nb_Freezing)/len(Index_nm_Freezing))
    RMM_freezing = len(Index_nb_Freezing)/8
    print(RSM_freezing)
    print(RMM_freezing)
    
    RSM_Prey = 1/(len(Index_nb_PC)/len(Index_nm_PC))
    RMM_Prey = len(Index_nb_PC)/8
    print(RSM_Prey)
    print(RMM_Prey)
    
    temp_df_fish = df_VMV_PVN[np.logical_and(df_VMV_PVN['Fish']==fish,df_VMV_PVN['Date']==int(date_no))]
    for plane_idx in range(1,10):
        temp_df_plane = temp_df_fish[temp_df_fish['Plane']==plane_idx]

        if temp_df_plane.shape[0]>0:
            for t in range(0,trial_no):
                temp_dir = main_dir+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane_idx)+'\\DeltaF_F\\'+fish+'_Trial'+str(t+1)+".csv"
                locals()['DeltaF_F_'+str(t)] = pd.read_csv(temp_dir)
            
            rs_Freezing = np.zeros(temp_df_plane.shape[0])
            for trial in Index_Freezing:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)

                rs_Freezing = rs_Freezing +(sti_on_delta_f>1).astype(int)
            
            rm_Freezing = np.zeros(temp_df_plane.shape[0])
            for trial in Index_nm_Freezing:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)
                rm_Freezing = rm_Freezing +(sti_on_delta_f>1).astype(int)
                
            rs_PC = np.zeros(temp_df_plane.shape[0])
            for trial in Index_Prey:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)
                rs_PC = rs_PC +(sti_on_delta_f>1).astype(int)
            
            rm_PC = np.zeros(temp_df_plane.shape[0])
            for trial in Index_nm_PC:
                all_ROI_id = locals()['DeltaF_F_'+str(trial)].ROI_id
                sorter = np.argsort(all_ROI_id)
                idx_ROI = sorter[np.searchsorted(all_ROI_id, temp_df_plane.ROI_id, sorter=sorter)].values
                
                delta_f = locals()['DeltaF_F_'+str(trial)].iloc[idx_ROI,:39].to_numpy()
                sti_on_delta_f = np.average(delta_f[:,27:36],axis=1)
                rm_PC = rm_PC +(sti_on_delta_f>1).astype(int)
            
            ratio_Freezing = (rs_Freezing/8)/(rm_Freezing/len(Index_nm_Freezing))
            ratio_Freezing[np.where(np.isinf(ratio_Freezing))[0]] = RSM_freezing # pure sensory, after normalization it would be 0
#             ratio_Freezing[np.where(np.isnan(ratio_Freezing))[0]] = RSM_freezing/2+RMM_freezing/2
#             ratio_Freezing[np.where(ratio_Freezing==0)[0]] = RSM_freezing
            
            ratio_PC = (rs_PC/8)/(rm_PC/len(Index_nm_PC))
            ratio_PC[np.where(np.isinf(ratio_PC))[0]] = RSM_Prey
#             ratio_PC[np.where(np.isnan(ratio_PC))[0]] = RSM_Prey/2+RMM_Prey/2
#             ratio_PC[np.where(ratio_PC==0)[0]] = RSM_Prey

            temp_df_plane['rm_freezing'] = rm_Freezing
            temp_df_plane['rs_freezing'] = rs_Freezing
            temp_df_plane['ratio_Freezing'] = ratio_Freezing
            temp_df_plane['normed_ratio_Freezing'] = (ratio_Freezing-RSM_freezing)/(RMM_freezing-RSM_freezing)
            temp_df_plane['normed_ratio_PC'] = (ratio_PC-RSM_Prey)/(RMM_Prey-RSM_Prey)
            df_active_PVN_normed_ratio_new = df_active_PVN_normed_ratio_new.append(temp_df_plane)
df_active_PVN_normed_ratio_new['cluster'] = clusters
df_active_PVN_normed_ratio_new.to_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_low_elevation\\df_normed_ratio_PVN_new.csv',index=False)

In [ ]:
df_active_PVN_normed_ratio_new['cluster'] = clusters
df_active_PVN_normed_ratio_new.to_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_low_elevation\\df_normed_ratio_PVN_new.csv',index=False)

In [ ]:
df_active_PVN_normed_ratio_new

In [ ]:

fig,ax1 = plt.subplots(1,1,figsize = (3.3,1),dpi=800)

temp_df = df_active_PVN_normed_ratio_new
cluster_idx = np.unique(temp_df.cluster)
scatter_info = np.zeros((len(cluster_idx),4))
temp_np = df_active_PVN_normed_ratio_new.iloc[:,:6].to_numpy()
for cluster_i in range(0,len(cluster_idx)):
    i = int(cluster_idx[cluster_i])
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(temp_df.cluster == i)[0])
    scatter_info[cluster_i,0] = np.average(temp_df.iloc[locals()['cluster_'+str(i)+'_roi_idx'],-2])
    average_temp_cluster = np.average(temp_np[locals()['cluster_'+str(i)+'_roi_idx'],:4],axis = 0)
    scatter_info[cluster_i,1] = (average_temp_cluster[2]-average_temp_cluster[1])/(average_temp_cluster[2]+average_temp_cluster[1])
    scatter_info[cluster_i,2] = len(locals()['cluster_'+str(i)+'_roi_idx'])
    scatter_info[cluster_i,3] = int(i)
scatter_info[:,2] = scatter_info[:,2]/np.sum(scatter_info[:,2])    
for cluster_i in range(0,len(cluster_idx)):  
    i = int(cluster_idx[cluster_i])
#     if i==100:
#         ax1.annotate(str(i),(scatter_info[cluster_i,1]-0.025,0),fontsize = 4)
    if scatter_info[cluster_i,2]>.04:
        ax1.annotate(str(i),(scatter_info[cluster_i,1]-0.015,0),fontsize = 4)
sc = plt.scatter(scatter_info[:,1],np.zeros(scatter_info.shape[0]),c = scatter_info[:,1],s = scatter_info[:,2]*1000,cmap = cm,vmin = -1,vmax = 1,alpha = 0.6)

sc_cbar = plt.colorbar(sc,shrink=1,extend = 'both',orientation = 'horizontal',location = 'top',pad = 0.05)
sc_cbar.set_label(label='Size Preference',size=12)
sc_cbar.set_ticks([-1,-0.5,0,0.5,1])
sc_cbar.set_ticklabels(['-1(4°)','-0.5','0','0.5','1(15°)'])
plt.xlabel('Size Preference Index',fontsize = 12)
plt.xlim(-1.05,1.05)
# plt.ylim(-0.5,1.5)
plt.yticks([0.0],['Tectum Cell Bodies'])
plt.show()

In [ ]:
# compare the sensory preference index between tectal neuropil pixels and PVN cell bodies
# version1:unit are pixel/cell body

df_active_pixel_normed_ratio = pd.read_csv('C:\\Users\\yxton\\Documents\\Code\\2p\\freezing_project\\clustering_files_RGC\\df_normed_ratio_ath5_new.csv')
# df_active_neuron_normed_ratio['Sensory_Index'][np.where(df_active_neuron_normed_ratio['Sensory_Index']<-1)[0]] = -1
# df_active_neuron_normed_ratio['Sensory_Index'][np.where(df_active_neuron_normed_ratio['Sensory_Index']>1)[0]] = 1

df_active_PVN_normed_ratio_new['Sensory_Index'] = (df_active_PVN_normed_ratio_new['Freezing']-df_active_PVN_normed_ratio_new['Prey'])/(df_active_PVN_normed_ratio_new['Freezing']+df_active_PVN_normed_ratio_new['Prey'])
df_active_pixel_normed_ratio['Sensory_Index'] = (df_active_pixel_normed_ratio['Freezing']-df_active_pixel_normed_ratio['Prey'])/(df_active_pixel_normed_ratio['Freezing']+df_active_pixel_normed_ratio['Prey'])

df_active_PVN_normed_ratio_new['Sensory_Index'][np.where(df_active_neuron_normed_ratio['Sensory_Index']<-1)[0]] = -1
df_active_PVN_normed_ratio_new['Sensory_Index'][np.where(df_active_neuron_normed_ratio['Sensory_Index']>1)[0]] = 1



df_active_PVN_normed_ratio_new['unit_type'] = 'Cell_Body'

df_sensory_idx = df_active_PVN_normed_ratio_new.iloc[:,-2:].append(df_active_pixel_normed_ratio.iloc[:,-5:-3])

fig,ax1 = plt.subplots(1,1,figsize = (6,6),dpi=800)
sns.violinplot(x='unit_type',y='Sensory_Index',data = df_sensory_idx,palette =['grey','lightgrey'],ax = ax1)

# plt.ylim(-1.3,1.3)
plt.ylim(-2,2)
plt.yticks([-1,-0.5,0,0.5,1],['-1(4°)','-0.5','0','0.5','1(15°)'],fontsize = 14)
plt.ylabel('Size Preference',fontsize = 16)
plt.xlabel('')
ax1.set_xticklabels(['Tectum Cell bodies','Tectal Neuropil pixels'],fontsize = 14)
plt.show()

In [ ]:
df_active_PVN_normed_ratio_new

In [ ]:
df_active_PVN_normed_ratio_new

In [ ]:
df_active_PVN_normed_ratio_new

In [ ]:
# compare the sensory preference index between tectal neuropil pixels and PVN cell bodies
# version1:unit are cluster


In [ ]:
# DBSCAN 
from sklearn.cluster import DBSCAN,OPTICS


In [ ]:
DBSCAN_result = OPTICS(min_samples=8).fit(npy_VMV_all)
DBSCAN_clusters = DBSCAN_result.labels_

In [ ]:
npy_VMV_all_cluster_DBSCAN = np.c_[npy_VMV_all, DBSCAN_clusters]

In [ ]:
np.max(DBSCAN_clusters)

In [ ]:
np.sum(DBSCAN_clusters == -1)

In [ ]:
DBSCAN_clusters.shape

In [ ]:
DBSCAN_clusters

In [ ]:
DBSCAN_result2 = DBSCAN(eps = 0.1,min_samples=5).fit(npy_VMV_all)
DBSCAN_clusters2 = DBSCAN_result2.labels_

In [ ]:
npy_VMV_all_cluster_DBSCAN2 = np.c_[npy_VMV_all, DBSCAN_clusters2]

In [ ]:
np.sum(DBSCAN_clusters2 == -1)

In [ ]:
np.max(DBSCAN_clusters2)

In [ ]:
# illustration
temp_score =np.zeros((1,6))
for i,j in zip([0.0,0.1,0.6,0.04,0.52,0.35],range(0,6)):
    temp_score[:,j] = i
    
fig,(ax1,ax2) = plt.subplots(1,2,figsize = (11,1),gridspec_kw=dict(width_ratios=[.95, .05]),dpi = 800)
sns.heatmap(temp_score,cmap = 'plasma',ax = ax1,vmin = -0.3,vmax = 0.7,cbar_ax = ax2)
visuomotor_list = ['No Stim', '4°', '15°', 'Luminance', 'Freezing (motor)', 'Prey (motor)']
ax1.set_xticklabels(visuomotor_list,fontsize = 13)
ax1.set_xlabel('Visuomotor Vectors',fontsize = 15)
ax1.set_yticks([]) 
ax2.set_ylabel('Averaged Score',fontsize = 14)
ax2.tick_params(labelsize=14)
# ax1.set_title('Visuomotor Vector of Example Neuron',fontsize = 16)

fig.subplots_adjust(wspace=0.02)
plt.show()

In [ ]:


for main_dir,fish in zip(main_dir_list,fish_index_list):
    date_no = main_dir.split('\\')[-1]
    for plane_i in range(1,plane_no+1):
        temp_df_coor = pd.DataFrame(columns = ['x','y','z','t','label'])

        slice_no = np.load(main_dir+'\\processed\\'+fish+'\\registration\\reg_2d\\plane'+str(plane_i)+'\\slice_no.npy')
        df_cell_info = pd.read_csv(main_dir+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane_i)+'\\DeltaF_F\\cell_info_'+fish+'.csv')

        for i in cluster_idx:
            temp_df_VMV = df_VMV_all.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:]
            for x in temp_df_VMV.index:
                if str(temp_df_VMV.loc[x,'Date'])==date_no and temp_df_VMV.loc[x,'Fish']==fish and temp_df_VMV.loc[x,'Plane']==plane_i:
                    cell_idx = np.where(df_cell_info.ROI_id == temp_df_VMV.loc[x,'ROI_id'])[0][0]
                    x_coor = np.array(df_cell_info.centroid_x[cell_idx])*509.12/339.41
                    y_coor = np.array(df_cell_info.centroid_y[cell_idx])*509.12/339.41
                    temp_df_coor_x = temp_df_coor.shape[0]
                    temp_df_coor.loc[temp_df_coor_x,:] = [x_coor,y_coor,slice_no,0,i]
        temp_df_coor.to_csv(main_dir+'\\processed\\'+fish+'\\registration\\reg_2d\\plane'+str(plane_i)+'\\'+fish[:2]+'_'+date_no+'_coordinates_v4.csv',index=False)

In [ ]:
temp_df_coor

In [ ]:
main_dir = 'D:\\LCr_UV_HSA_Luminance_4\\'
cluster_idx = cluster_7_roi_idx+cluster_9_roi_idx+cluster_10_roi_idx

temp_df_ROI_info = df_ROI_info_all.iloc[cluster_idx,:]
for date_no in np.unique(temp_df_ROI_info.Date):
    temp_df_ROI_info_date = temp_df_ROI_info[temp_df_ROI_info.Date==date_no]
    for fish in np.unique(temp_df_ROI_info_date.Fish):
        temp_df_ROI_info_fish = temp_df_ROI_info_date[temp_df_ROI_info_date.Fish==fish]
        for plane in np.unique(temp_df_ROI_info_fish.Plane):
            temp_df_ROI_info_plane = temp_df_ROI_info_fish[temp_df_ROI_info_fish.Plane==plane]
            df_cell_info = pd.read_csv(main_dir+date_no+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane)+'\\DeltaF_F\\cell_info_'+fish+'.csv')
            
            temp_cell_idx_list = []
            for ROI_id in temp_df_ROI_info_plane.ROI_id:
                temp_cell_idx_list.append(np.where(df_cell_info.ROI_id == ROI_id)[0][0])
                
            x_coor = np.array(df_cell_info.centroid_x[temp_cell_idx_list])
            y_coor = np.array(df_cell_info.centroid_y[temp_cell_idx_list])
            radius_list = np.array(df_cell_info.radius[temp_cell_idx_list])
            
            
            file = main_dir+date_no+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane)+'\\plane0\\reg_tif\\file000_chan0.tif'
            temp_tif = tifffile.imread(file)
#             temp_average_tif = np.average(temp_tif,axis = (1,2))
        #     print(temp_average_tif.shape)
#             temp_frame = np.argmax(temp_average_tif)
            plt.figure(figsize = (5,5))
            ax_baseline = sns.heatmap(temp_tif, cmap='gray', vmin=0, vmax=255,square = True,cbar = False)
            ax_baseline.scatter(x_coor,y_coor,radius_list*1.5,c = 'r')
            ax_baseline.tick_params(left=False, bottom=False)
            ax_baseline.axis('off')
            plt.title(fish[:2]+' '+date_no+' Plane'+str(plane))
            plt.show()

In [ ]:
df_VMV_all.iloc[locals()['cluster_'+str(i)+'_roi_idx'],:]

In [ ]:
df_VMV_all

In [ ]:
np.add(temp_averaged_score,np.array(listr2) * np.array(listr2))

In [ ]:
# 20230102 F2

main_dir = "D:\\LCr_UV_HSA_Luminance_4\\20230102"
date_no = main_dir.split('\\')[-1]
fish_index_list =['F2_16_00']

stim_list = ['ns','Prey','Freezing','Luminance']
time_len = 39
plane_no = 9

for i in range(0,len(fish_index_list)):   
    # vsinfo
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish_index_list[i]+'//vsinfo_'+date_no+'_'+fish_index_list[i]+'.xlsx')
    trial_no = vsinfo.shape[0]
    stim_trial_idx = [i+1 for i in np.where(vsinfo.Left_Stimulus_Type=='m')[0]]+[i+1 for i in np.where(vsinfo.Right_Stimulus_Type=='m')[0]]
    
    # every trial's data
    for plane_i in range(1,plane_no+1):
        for t in range(1,trial_no+1):
            locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)+'_dir'] = main_dir+'\\processed\\'+fish_index_list[i]+'\\processed_suite2p\\plane'+str(plane_i)+'\\DeltaF_F\\'+fish_index_list[i]+'_Trial'+str(t)+".csv"
            print(locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)+'_dir'])
            locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)] = pd.read_csv(locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)+'_dir'])


        # roi number
        locals()['roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)] = locals()['df_'+date_no+'_'+str(fish_index_list[i])+'_p'+str(plane_i)+'_trial_1'].shape[0]
        print(date_no + str(fish_index_list[i])+' plane'+str(plane_i)+' roi number is ' + str(locals()['roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)]))

        locals()['active_roi_idx_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)] = list()
        for roi_i in range(0,locals()['roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)]):
            temp_activity = np.zeros([len(stim_trial_idx),time_len])
            temp_activity_normed = np.zeros([len(stim_trial_idx),time_len])
            temp_active_no = 0
            for j in range(0,len(stim_trial_idx)):
                temp_activity[j] = locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(stim_trial_idx[j])].iloc[roi_i,range(0,time_len)].values
                if np.max(temp_activity[j])>0.2:
                    temp_activity_normed[j] = (temp_activity[j]-min(temp_activity[j]))/(max(temp_activity[j])-min(temp_activity[j]))
            listregintercept,listregcoeff,listr2 = get_regression(temp_activity_normed)
            for temp_i in range(0,len(listr2)):
                if  listregcoeff[temp_i] > 0.5 and listr2[temp_i]>0.5:
                    
                    temp_active_no = temp_active_no+1
    #         print(temp_active_no)
            if temp_active_no>=3:
                locals()['active_roi_idx_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)].append(roi_i)
                for temp_i in range(0,len(listr2)):
                    if  listregcoeff[temp_i] > 0.5 and listr2[temp_i]>0.6:
                        plt.plot(temp_activity_normed[temp_i])
                        plt.show()
                        print('roi no. is '+str(roi_i))
                        print('coeffcient is '+str(listregcoeff[temp_i]))
                        print('R2 is '+str(listr2[temp_i]))
        for t in range(1,trial_no+1):
            locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)] = locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)].iloc[locals()['active_roi_idx_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)],:]

         # active roi number
        locals()['active_roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)] = locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)].shape[0]
        print(date_no + str(fish_index_list[i])+' plane'+str(plane_i)+' active roi number is ' + str(locals()['active_roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)])) 



In [ ]:
stim_list = ['ns','Prey','Freezing','Luminance']
    

for i in range(0,len(stim_list)):
# for i in range(1,2): 
    list_trace_temp = list()
    list_info_temp = list()
    for fish_i in range(0,len(fish_index_list)):
        vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish_index_list[fish_i]+'//vsinfo_'+date_no+'_'+fish_index_list[fish_i]+'.xlsx')
        trial_no = vsinfo.shape[0]
        
        for ii in range(0,len(stim_list)):
            locals()['Index_'+str(stim_list[ii])] = []
        for t in range(0,trial_no):
            if vsinfo.Left_Stimulus_Type[t] == 'n' and vsinfo.Right_Stimulus_Type[t] == 'n':
                Index_ns.append(t+1)
            if vsinfo.Left_Stimulus_Type[t] == 'lum' or vsinfo.Right_Stimulus_Type[t] == 'lum':
                Index_Luminance.append(t+1)
            if vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 15:
                    Index_Freezing.append(t+1)
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 4:
                    Index_Prey.append(t+1)
        index_temp = locals()['Index_'+str(stim_list[i])]
        print(index_temp)
        for plane_i in range(1,plane_no+1):
            locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] = pd.DataFrame(columns = locals()['df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_1'].iloc[:,range(0,time_len)].columns, index = locals()['df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_1'].iloc[:,range(0,time_len)].index)
            locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].loc[:,:] = 0
            len_temp = len(index_temp)
            #####
            print(index_temp)
            print(['df_averaged_'+date_no+fish_index_list[fish_i] +' plane'+str(plane_i) +' is'])
            print(locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)])
            ######
            for t in index_temp:
                # then we need to average all the rows of dfs in this index
                df_name_temp = 'df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_'+str(t)
                # debug part
    #             print(locals()[df_name_temp].iloc[:,range(2,94)].shape)
    #             print(locals()['df_averaged_'+fish_index_list[fish_i]])
                locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] = locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] + locals()[df_name_temp].iloc[:,range(0,time_len)]/len_temp
            
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] = pd.DataFrame(columns = ['Date','Fish','Plane','ROI_id'])
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].ROI_id = locals()['df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_'+str(index_temp[0])].ROI_id
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].Date = date_no
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].Fish = fish_index_list[fish_i]
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].Plane = plane_i
            
            list_trace_temp.append(locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)])
            list_info_temp.append(locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)])
    locals()['df_averaged_stim_'+date_no+'_'+str(stim_list[i])] = pd.concat(list_trace_temp)
    locals()['df_ROI_info_'+date_no] = pd.concat(list_info_temp)
    

# concat different stimulus's data
df_averaged_stim_list = []
for i in range(0,len(stim_list)):
    df_averaged_stim_list.append(locals()['df_averaged_stim_'+date_no+'_'+str(stim_list[i])])
Jan2_clustering_data = pd.concat(df_averaged_stim_list, axis=1)

locals()['df_ROI_info_'+date_no].index = range(0,locals()['df_ROI_info_'+date_no].shape[0])
Jan2_clustering_data.index = range(0,Jan2_clustering_data.shape[0])


# npy_clustering_data = clustering_data.to_numpy(dtype = 'float64')
# npy_clustering_data.shape

In [ ]:
#20230109 F3 F4

main_dir = "D:\\LCr_UV_HSA_Luminance_5\\20230109"
date_no = main_dir.split('\\')[-1]
fish_index_list =['F3_20_30','F4_22_30']

stim_list = ['ns','Prey','Freezing','Luminance']
time_len = 39
plane_no = 9

for i in range(0,len(fish_index_list)):   
    # vsinfo
    vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish_index_list[i]+'//vsinfo_'+date_no+'_'+fish_index_list[i]+'.xlsx')
    trial_no = vsinfo.shape[0]
    stim_trial_idx = [i+1 for i in np.where(vsinfo.Left_Stimulus_Type=='m')[0]]+[i+1 for i in np.where(vsinfo.Right_Stimulus_Type=='m')[0]]
    
    # every trial's data
    for plane_i in range(1,plane_no+1):
        for t in range(1,trial_no+1):
            locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)+'_dir'] = main_dir+'\\processed\\'+fish_index_list[i]+'\\processed_suite2p\\plane'+str(plane_i)+'\\DeltaF_F\\'+fish_index_list[i]+'_Trial'+str(t)+".csv"
            print(locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)+'_dir'])
            locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)] = pd.read_csv(locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)+'_dir'])


        # roi number
        locals()['roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)] = locals()['df_'+date_no+'_'+str(fish_index_list[i])+'_p'+str(plane_i)+'_trial_1'].shape[0]
        print(date_no + str(fish_index_list[i])+' plane'+str(plane_i)+' roi number is ' + str(locals()['roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)]))

        locals()['active_roi_idx_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)] = list()
        for roi_i in range(0,locals()['roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)]):
            temp_activity = np.zeros([len(stim_trial_idx),time_len])
            temp_activity_normed = np.zeros([len(stim_trial_idx),time_len])
            temp_active_no = 0
            for j in range(0,len(stim_trial_idx)):
                temp_activity[j] = locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(stim_trial_idx[j])].iloc[roi_i,range(0,time_len)].values
                if np.max(temp_activity[j])>0.2:
                    temp_activity_normed[j] = (temp_activity[j]-min(temp_activity[j]))/(max(temp_activity[j])-min(temp_activity[j]))
            listregintercept,listregcoeff,listr2 = get_regression(temp_activity_normed)
            for temp_i in range(0,len(listr2)):
                if  listregcoeff[temp_i] > 0.5 and listr2[temp_i]>0.6:
                    temp_active_no = temp_active_no+1
    #         print(temp_active_no)
            if temp_active_no>=3:
                locals()['active_roi_idx_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)].append(roi_i)
        for t in range(1,trial_no+1):
            locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)] = locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)].iloc[locals()['active_roi_idx_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)],:]

         # active roi number
        locals()['active_roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)] = locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)].shape[0]
        print(date_no + str(fish_index_list[i])+' plane'+str(plane_i)+' active roi number is ' + str(locals()['active_roi_num_'+date_no+str(fish_index_list[i])+'_p'+str(plane_i)])) 


In [ ]:
stim_list = ['ns','Prey','Freezing','Luminance']
    

for i in range(0,len(stim_list)):
# for i in range(1,2): 
    list_trace_temp = list()
    list_info_temp = list()
    for fish_i in range(0,len(fish_index_list)):
        vsinfo = pd.read_excel(main_dir+'\\behavior\\'+fish_index_list[fish_i]+'//vsinfo_'+date_no+'_'+fish_index_list[fish_i]+'.xlsx')
        trial_no = vsinfo.shape[0]
        
        for ii in range(0,len(stim_list)):
            locals()['Index_'+str(stim_list[ii])] = []
        for t in range(0,trial_no):
            if vsinfo.Left_Stimulus_Type[t] == 'n' and vsinfo.Right_Stimulus_Type[t] == 'n':
                Index_ns.append(t+1)
            if vsinfo.Left_Stimulus_Type[t] == 'lum' or vsinfo.Right_Stimulus_Type[t] == 'lum':
                Index_Luminance.append(t+1)
            if vsinfo.Left_Stimulus_Type[t] == 'm' or vsinfo.Right_Stimulus_Type[t] == 'm':
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 15:
                    Index_Freezing.append(t+1)
                if vsinfo.Left_Stimulus_Size[t] + vsinfo.Right_Stimulus_Size[t] == 4:
                    Index_Prey.append(t+1)
        index_temp = locals()['Index_'+str(stim_list[i])]
        print(index_temp)
        for plane_i in range(1,plane_no+1):
            locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] = pd.DataFrame(columns = locals()['df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_1'].iloc[:,range(0,time_len)].columns, index = locals()['df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_1'].iloc[:,range(0,time_len)].index)
            locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].loc[:,:] = 0
            len_temp = len(index_temp)
            #####
            print(index_temp)
            print(['df_averaged_'+date_no+fish_index_list[fish_i] +' plane'+str(plane_i) +' is'])
            print(locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)])
            ######
            for t in index_temp:
                # then we need to average all the rows of dfs in this index
                df_name_temp = 'df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_'+str(t)
                # debug part
    #             print(locals()[df_name_temp].iloc[:,range(2,94)].shape)
    #             print(locals()['df_averaged_'+fish_index_list[fish_i]])
                locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] = locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] + locals()[df_name_temp].iloc[:,range(0,time_len)]/len_temp
            
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)] = pd.DataFrame(columns = ['Date','Fish','Plane','ROI_id'])
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].ROI_id = locals()['df_'+date_no+'_'+fish_index_list[fish_i]+'_p'+str(plane_i)+'_trial_'+str(index_temp[0])].ROI_id
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].Date = date_no
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].Fish = fish_index_list[fish_i]
            locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)].Plane = plane_i
            
            list_trace_temp.append(locals()['df_averaged_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)])
            list_info_temp.append(locals()['df_ROI_info_'+date_no+fish_index_list[fish_i]+'_p'+str(plane_i)])
    locals()['df_averaged_stim_'+date_no+'_'+str(stim_list[i])] = pd.concat(list_trace_temp)
    locals()['df_ROI_info_'+date_no] = pd.concat(list_info_temp)
    

# concat different stimulus's data
df_averaged_stim_list = []
for i in range(0,len(stim_list)):
    df_averaged_stim_list.append(locals()['df_averaged_stim_'+date_no+'_'+str(stim_list[i])])
Jan9_clustering_data = pd.concat(df_averaged_stim_list, axis=1)

locals()['df_ROI_info_'+date_no].index = range(0,locals()['df_ROI_info_'+date_no].shape[0])
Jan9_clustering_data.index = range(0,Jan9_clustering_data.shape[0])



In [ ]:
npy_clustering_data = Jan9_clustering_data.to_numpy(dtype = 'float64')
npy_clustering_data.shape

In [ ]:
clustering_data_all = pd.concat([Jan2_clustering_data,Jan9_clustering_data], axis=0)
df_ROI_info_all = pd.concat([df_ROI_info_20230102,df_ROI_info_20230109],axis=0)

npy_clustering_data = clustering_data_all.to_numpy(dtype = 'float64')
npy_clustering_data.shape

In [ ]:
# npy_clustering_data = clustering_data.to_numpy(dtype = 'float64')
corr = np.corrcoef(npy_clustering_data)
print(np.amin(corr))
sns.heatmap(corr)

In [ ]:
corr = np.corrcoef(npy_clustering_data)

fig, axes = plt.subplots(1, 2, figsize=(9, 8), gridspec_kw=dict(width_ratios=[.8, .05]))

sns.heatmap(corr, vmax=1, vmin=-1, cmap='bwr', ax=axes[0], cbar_ax=axes[1], square=True,
            cbar_kws=dict(ticks=[0, 0.5, 1], label='r'))

for ax in axes.ravel()[:-1]:
    ax.set_yticks([])
    ax.set_xticks([])

axes[0].set_xlabel('ROI')

fig.subplots_adjust(wspace=0)

In [ ]:
threshold_ratio = 0.3
d = sch.distance.pdist(corr)
L = sch.linkage(d, method='complete')
clusters = sch.fcluster(L, threshold_ratio * d.max(), 'distance')

argsort = np.argsort(clusters)[::-1]

fig, axes = plt.subplots(1, 3, figsize=(11, 8), gridspec_kw=dict(width_ratios=[.2, .8, .05]))
sch.dendrogram(L, orientation='left', ax=axes[0], color_threshold=threshold_ratio*d.max())
sns.heatmap(corr[argsort][:, argsort], vmax=1, vmin=-1, cmap='bwr', ax=axes[1], cbar_ax=axes[2],
            square=True, cbar_kws=dict(ticks=[-1,0,1], label='correlation'))

axes[0].axis('off')
axes[1].set_yticks([])
axes[1].set_xticks([])
axes[1].set_ylabel('ROI')


# fig.suptitle("Hierechical Clustering of stimulus active neurons(n=598 in 9 fish)",fontsize = 20)
fig.subplots_adjust(wspace=0)

In [ ]:
cluster_idx=np.unique(clusters)
for i in cluster_idx:
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])

In [ ]:
def Plot_Overlaid_Activity(ROI_iloc_list):
#     locals()[Fish_Index +'Max_DeltaF'] = pandas.read_excel(Saving_Directory + Fish_Index + '_Max_DeltaF2' + '.xlsx',sheet_name='Max_F')
#     locals()[Fish_Index +'Max_DeltaF'].drop(locals()[Fish_Index +'Max_DeltaF'].columns[0], inplace=True, axis=1)

#     Stimulus_Sizes = [2,3,4,6,8,10,0]
#     selective_clustering_data
    ROI_Activity_AVG = []
    ROI_Activity_STD = []
#         ROI_Activity = pandas.DataFrame()
#         for n in ROI_list:
#             if vsinfo.Right[n] == Stimulus_Sizes[s]:
#                 Trial = pandas.read_excel(Saving_Directory + Fish_Index + '_Trial' + str(n+1) + '.xlsx')
#                 ROI_Activity = ROI_Activity.append(Trial[Trial.Label == ROI])
    total_frame_no = npy_clustering_data.shape[1]
    for frame in range(0,total_frame_no):
        ROI_Activity_AVG.append(npy_clustering_data[ROI_iloc_list, frame].mean())
        ROI_Activity_STD.append(npy_clustering_data[ROI_iloc_list, frame].std())

#     # plot
#     Onset = 39
#     move = 46
#     stop = 54

    fig, ax = plt.subplots(1, 1, figsize=(15, 3))
#     plt.title('Averged Activity of ROI ' + str(ROI) + ' to different dot sizes', fontsize=20)
#     ax.set_ylabel('DeltaF/F', fontsize=15)
#     ax.set_xlabel('Frames', fontsize=16)
    ax.set_ylabel('DeltaF/F', fontsize=16)
    plt.ylim(-0.2, 3)
    plt.xlim(0, 39*4)
    plt.axvline(39, color='grey',alpha = 0.3)
    plt.axvline(39*2, color='grey',alpha = 0.3)
    plt.axvline(39*3, color='grey',alpha = 0.3)

    
#     plt.fill_between(np.arange(27,30),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39,30+39),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*2,30+39*2),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*3,30+39*3),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    
#     plt.axvline(Onset, 0, 100, label='Onset', color='grey')
#     plt.axvline(move, 0, 100, label='Move', color='red')
#     plt.axvline(stop, 0, 100, label='Stop', color='grey')
#     plt.axvline(stop + 8, 0, 100, label='1s after Stop', color='orange')
#     plt.axhline(0, 0, 121, label='AVG_Baseline', color='purple')
#     ax.tick_params(axis='x', labelsize=15)
#     ax.tick_params(axis='y', labelsize=15)
#     x_ticks = numpy.arange(0, 92, 7.7)
#     plt.xticks(x_ticks)
    

    ROI_Activity_AVG = np.array(ROI_Activity_AVG)
    ROI_Activity_STD = np.array(ROI_Activity_STD)
    yhat = ROI_Activity_AVG
    y_minus = ROI_Activity_AVG - ROI_Activity_STD
    y_plus = ROI_Activity_AVG + ROI_Activity_STD
#     bp = ax.plot(yhat, color='C',  linewidth=3)
    for i in ROI_iloc_list:
        ax.plot(npy_clustering_data[i,:],color = 'silver',linewidth=0.4,alpha = 0.25)
    plt.fill_between(range(0, total_frame_no), y_minus, y_plus, alpha=0.3, facecolor='c')
    bp = ax.plot(yhat, color='c',  linewidth=3)
#     bp = ax.plot(locals()['ROI_Activity_AVG_'+ str(Stimulus_Sizes[6])], color='C', label= str(Stimulus_Sizes[6])+ ' Degree', linewidth=3)
    
#     if locals()[Fish_Index +'Max_DeltaF'][locals()[Fish_Index +'Max_DeltaF'].Label == ROI].Side_is_Left.tolist()[0] == True:
#         Side = 'Ipsilateral to Stimulus'
#     else:
#         Side = 'Contralateral to Stimulus'
    
#     plt.plot([0],[0], color='black', label = 'Side = ' + Side )
#     plt.plot([0],[0], color='black', label = 'Size_Preference = ' + str(format(locals()[Fish_Index +'Max_DeltaF'][locals()[Fish_Index +'Max_DeltaF'].Label == ROI].Size_Pref_Index.tolist()[0],'.2f')))
#     plt.legend()
#     plt.setp(ax.get_xticklabels(), visible=False)
#     plt.setp(ax.get_yticklabels(), visible=False)
    plt.xticks([19,58,97,136])
    plt.yticks(fontsize = 14)
    ax.set_xticklabels(['No stimulus','Prey stimulus','Freezing stimulus','Luminance Control'],fontsize = 16)
    ax.tick_params(axis='x', which='both', length=0)
#     ax.axis('off')
#     plt.show()
    return yhat

In [ ]:
for i in cluster_idx:
    Plot_Overlaid_Activity(locals()['cluster_'+str(i)+'_roi_idx'])
    plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 16)
    plt.show()

In [ ]:
df_ROI_info_20230102

In [ ]:
df_ROI_info_all

In [ ]:
np_cluster_19_id = np.array([ 276,  558, 1211, 1660,  343,  624,  840, 1404, 1411,  238,  740,
       1138, 1301, 1407, 1976,   40,  127,  608,  819, 1321, 1545, 1784,
       1845, 2036, 2044, 2252, 2259, 2536,   97,  284, 1503, 2589])
# plane
for ROI in np_cluster_19_id:
    print(df_ROI_info_all[df_ROI_info_all.ROI_id == ROI])


In [ ]:
npy_clustering_data = clustering_data_all.to_numpy(dtype = 'float64')
corr = np.corrcoef(npy_clustering_data)
print(np.amin(corr))
sns.heatmap(corr)

In [ ]:
corr = np.corrcoef(npy_clustering_data)

fig, axes = plt.subplots(1, 2, figsize=(9, 8), gridspec_kw=dict(width_ratios=[.8, .05]))

sns.heatmap(corr, vmax=1, vmin=-1, cmap='bwr', ax=axes[0], cbar_ax=axes[1], square=True,
            cbar_kws=dict(ticks=[0, 0.5, 1], label='r'))

for ax in axes.ravel()[:-1]:
    ax.set_yticks([])
    ax.set_xticks([])

axes[0].set_xlabel('ROI')

fig.subplots_adjust(wspace=0)

In [ ]:
threshold_ratio = 0.25
d = sch.distance.pdist(corr)
L = sch.linkage(d, method='complete')
clusters = sch.fcluster(L, threshold_ratio * d.max(), 'distance')

argsort = np.argsort(clusters)[::-1]

fig, axes = plt.subplots(1, 3, figsize=(11, 8), gridspec_kw=dict(width_ratios=[.2, .8, .05]))
sch.dendrogram(L, orientation='left', ax=axes[0], color_threshold=threshold_ratio*d.max())
sns.heatmap(corr[argsort][:, argsort], vmax=1, vmin=-1, cmap='bwr', ax=axes[1], cbar_ax=axes[2],
            square=True, cbar_kws=dict(ticks=[-1,0,1], label='correlation'))

axes[0].axis('off')
axes[1].set_yticks([])
axes[1].set_xticks([])
axes[1].set_ylabel('ROI')


# fig.suptitle("Hierechical Clustering of stimulus active neurons(n=598 in 9 fish)",fontsize = 20)
fig.subplots_adjust(wspace=0)

In [ ]:
cluster_idx=np.unique(clusters)
for i in cluster_idx:
    locals()['cluster_'+str(i)+'_roi_idx'] = list(np.where(clusters == i)[0])

In [ ]:
def Plot_Overlaid_Activity(ROI_iloc_list):
#     locals()[Fish_Index +'Max_DeltaF'] = pandas.read_excel(Saving_Directory + Fish_Index + '_Max_DeltaF2' + '.xlsx',sheet_name='Max_F')
#     locals()[Fish_Index +'Max_DeltaF'].drop(locals()[Fish_Index +'Max_DeltaF'].columns[0], inplace=True, axis=1)

#     Stimulus_Sizes = [2,3,4,6,8,10,0]
#     selective_clustering_data
    ROI_Activity_AVG = []
    ROI_Activity_STD = []
#         ROI_Activity = pandas.DataFrame()
#         for n in ROI_list:
#             if vsinfo.Right[n] == Stimulus_Sizes[s]:
#                 Trial = pandas.read_excel(Saving_Directory + Fish_Index + '_Trial' + str(n+1) + '.xlsx')
#                 ROI_Activity = ROI_Activity.append(Trial[Trial.Label == ROI])
    total_frame_no = npy_clustering_data.shape[1]
    for frame in range(0,total_frame_no):
        ROI_Activity_AVG.append(npy_clustering_data[ROI_iloc_list, frame].mean())
        ROI_Activity_STD.append(npy_clustering_data[ROI_iloc_list, frame].std())

#     # plot
#     Onset = 39
#     move = 46
#     stop = 54

    fig, ax = plt.subplots(1, 1, figsize=(15, 3))
#     plt.title('Averged Activity of ROI ' + str(ROI) + ' to different dot sizes', fontsize=20)
#     ax.set_ylabel('DeltaF/F', fontsize=15)
#     ax.set_xlabel('Frames', fontsize=16)
    ax.set_ylabel('DeltaF/F', fontsize=16)
    plt.ylim(-0.2, 3)
    plt.xlim(0, 39*4)
    plt.axvline(39, color='grey',alpha = 0.3)
    plt.axvline(39*2, color='grey',alpha = 0.3)
    plt.axvline(39*3, color='grey',alpha = 0.3)

    
#     plt.fill_between(np.arange(27,30),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39,30+39),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*2,30+39*2),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    plt.fill_between(np.arange(27+39*3,30+39*3),3,-0.2,linewidth = 1,color = 'lightcoral',alpha = 0.2)
    
#     plt.axvline(Onset, 0, 100, label='Onset', color='grey')
#     plt.axvline(move, 0, 100, label='Move', color='red')
#     plt.axvline(stop, 0, 100, label='Stop', color='grey')
#     plt.axvline(stop + 8, 0, 100, label='1s after Stop', color='orange')
#     plt.axhline(0, 0, 121, label='AVG_Baseline', color='purple')
#     ax.tick_params(axis='x', labelsize=15)
#     ax.tick_params(axis='y', labelsize=15)
#     x_ticks = numpy.arange(0, 92, 7.7)
#     plt.xticks(x_ticks)
    

    ROI_Activity_AVG = np.array(ROI_Activity_AVG)
    ROI_Activity_STD = np.array(ROI_Activity_STD)
    yhat = ROI_Activity_AVG
    y_minus = ROI_Activity_AVG - ROI_Activity_STD
    y_plus = ROI_Activity_AVG + ROI_Activity_STD
#     bp = ax.plot(yhat, color='C',  linewidth=3)
    for i in ROI_iloc_list:
        ax.plot(npy_clustering_data[i,:],color = 'silver',linewidth=0.4,alpha = 0.25)
    plt.fill_between(range(0, total_frame_no), y_minus, y_plus, alpha=0.3, facecolor='c')
    bp = ax.plot(yhat, color='c',  linewidth=3)
#     bp = ax.plot(locals()['ROI_Activity_AVG_'+ str(Stimulus_Sizes[6])], color='C', label= str(Stimulus_Sizes[6])+ ' Degree', linewidth=3)
    
#     if locals()[Fish_Index +'Max_DeltaF'][locals()[Fish_Index +'Max_DeltaF'].Label == ROI].Side_is_Left.tolist()[0] == True:
#         Side = 'Ipsilateral to Stimulus'
#     else:
#         Side = 'Contralateral to Stimulus'
    
#     plt.plot([0],[0], color='black', label = 'Side = ' + Side )
#     plt.plot([0],[0], color='black', label = 'Size_Preference = ' + str(format(locals()[Fish_Index +'Max_DeltaF'][locals()[Fish_Index +'Max_DeltaF'].Label == ROI].Size_Pref_Index.tolist()[0],'.2f')))
#     plt.legend()
#     plt.setp(ax.get_xticklabels(), visible=False)
#     plt.setp(ax.get_yticklabels(), visible=False)
    plt.xticks([19,58,97,136])
    plt.yticks(fontsize = 14)
    ax.set_xticklabels(['No stimulus','Prey stimulus','Freezing stimulus','Luminance Control'],fontsize = 16)
    ax.tick_params(axis='x', which='both', length=0)
#     ax.axis('off')
#     plt.show()
    return yhat

In [ ]:
for i in cluster_idx:
    Plot_Overlaid_Activity(locals()['cluster_'+str(i)+'_roi_idx'])
    plt.title('Averaged activity of cluster'+str(i)+', n = '+str(len(locals()['cluster_'+str(i)+'_roi_idx'])),fontsize = 16)
    plt.show()

In [ ]:
for i in idx_freezing_cluster:
    print(clusters[i])

In [ ]:
for i in idx_freezing_cluster:
#     if clusters[i] == 6 or clusters[i] == 8:
    if clusters[i] == 9:
        fig, ax = plt.subplots(1, 1, figsize=(15, 3))
        plt.ylim(-0.2, 3)
        plt.xlim(0, 39*4)
        plt.axvline(39, color='grey',alpha = 0.3)
        plt.axvline(39*2, color='grey',alpha = 0.3)
        plt.axvline(39*3, color='grey',alpha = 0.3)
        ax.plot(range(0,39*4),npy_clustering_data[i,:],color = 'C',linewidth=2)

In [ ]:
cluster_9_roi_idx

In [ ]:
for i in range(0,len(cluster_10_roi_idx)):
    fig, ax = plt.subplots(1, 1, figsize=(15, 3))
    plt.ylim(-0.2, 3)
    plt.xlim(0, 39*4)
    plt.axvline(39, color='grey',alpha = 0.3)
    plt.axvline(39*2, color='grey',alpha = 0.3)
    plt.axvline(39*3, color='grey',alpha = 0.3)
    ax.plot(range(0,39*4),npy_clustering_data[cluster_10_roi_idx[i],:],color = 'C',linewidth=2)
#     total_frame_no = npy_3_selective_double_data.shape[1]
#     ax.plot(range(92, total_frame_no+92),npy_3_selective_double_data[cluster_3_roi_idx[i],:],color = 'r',linewidth=2)
#     ax.plot(range(92, total_frame_no+92),npy_6_selective_double_data[cluster_3_roi_idx[i],:],color = 'y',linewidth=2)
#     ax.plot(range(92, total_frame_no+92),npy_10_selective_double_data[cluster_3_roi_idx[i],:],color = 'b',linewidth=2)

#     # ax.axis('off')
    plt.show()

In [ ]:
main_dir = 'D:\\LCr_UV_HSA_Luminance_4\\'
cluster_idx = cluster_7_roi_idx+cluster_9_roi_idx+cluster_10_roi_idx

temp_df_ROI_info = df_ROI_info_all.iloc[cluster_idx,:]
for date_no in np.unique(temp_df_ROI_info.Date):
    temp_df_ROI_info_date = temp_df_ROI_info[temp_df_ROI_info.Date==date_no]
    for fish in np.unique(temp_df_ROI_info_date.Fish):
        temp_df_ROI_info_fish = temp_df_ROI_info_date[temp_df_ROI_info_date.Fish==fish]
        for plane in np.unique(temp_df_ROI_info_fish.Plane):
            temp_df_ROI_info_plane = temp_df_ROI_info_fish[temp_df_ROI_info_fish.Plane==plane]
            df_cell_info = pd.read_csv(main_dir+date_no+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane)+'\\DeltaF_F\\cell_info_'+fish+'.csv')
            
            temp_cell_idx_list = []
            for ROI_id in temp_df_ROI_info_plane.ROI_id:
                temp_cell_idx_list.append(np.where(df_cell_info.ROI_id == ROI_id)[0][0])
                
            x_coor = np.array(df_cell_info.centroid_x[temp_cell_idx_list])
            y_coor = np.array(df_cell_info.centroid_y[temp_cell_idx_list])
            radius_list = np.array(df_cell_info.radius[temp_cell_idx_list])
            
            
            file = main_dir+date_no+'\\processed\\'+fish+'\\processed_suite2p\\plane'+str(plane)+'\\plane0\\reg_tif\\file000_chan0.tif'
            temp_tif = tifffile.imread(file)
#             temp_average_tif = np.average(temp_tif,axis = (1,2))
        #     print(temp_average_tif.shape)
#             temp_frame = np.argmax(temp_average_tif)
            plt.figure(figsize = (5,5))
            ax_baseline = sns.heatmap(temp_tif, cmap='gray', vmin=0, vmax=255,square = True,cbar = False)
            ax_baseline.scatter(x_coor,y_coor,radius_list*1.5,c = 'r')
            ax_baseline.tick_params(left=False, bottom=False)
            ax_baseline.axis('off')
            plt.title(fish[:2]+' '+date_no+' Plane'+str(plane))
            plt.show()

In [ ]:
temp_df_ROI_info_plane

In [ ]:
df_cell_info.iloc[temp_cell_idx_list,:]
np.array(df_cell_info.centroid_x[temp_cell_idx_list])

In [ ]:
df_cell_info

In [ ]:
cluster_7_roi_idx

In [ ]:
range(0,len(fish_index_list))

In [ ]:
locals()['df_'+date_no+'_'+fish_index_list[i]+'_p'+str(plane_i)+'_trial_'+str(t)]